In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pathlib
import shutil
import os
import requests

curr_dir = pathlib.Path(".")
data_dir = curr_dir / "RTS-GMLC-master" / "RTS_Data"
wind_data_dir = curr_dir / "wind-data"

In [3]:
# Read in bus geodata and generation data for IEEE RTS GMLC (Reliability Test System)
df_bus = pd.read_csv(data_dir / "SourceData" / "bus.csv", index_col=[0])
df_geodata = df_bus[["lat", "lng"]]
df_gen_full = pd.read_csv(data_dir / "SourceData" / "gen.csv", index_col=[0])

In [4]:
# Get wind geodata
wind_geodata = df_geodata.loc[df_gen_full.loc[df_gen_full["Unit Type"] == "WIND", "Bus ID"]]

In [5]:
wind_geodata

,lat,lng
Bus ID,,
309,34.735758,-118.127342
317,35.378433,-117.055829
303,35.217543,-118.043550
122,32.818864,-116.324347


In [29]:
# Read in API key
with open("api-key.txt", "r") as f:
    API_KEY = f.readlines()[0].strip()

# Build request

url = (
    "https://developer.nrel.gov/api/wind-toolkit/v2/wind/"
    "wtk-now23-california-v1-0-0-5min-download.csv"
    f"?api_key={API_KEY}"
)

for bus_id, row in wind_geodata.iterrows():

    print(f"Retrieving wind data for bus {bus_id}...")
    
    # Create directory for wind data for all years
    bus_wind_data_dir = wind_data_dir / f"bus{bus_id}"
    bus_wind_data_dir.mkdir(exist_ok=True, parents=True)
    
    # Extract bus geodata for request
    lat, lon = row.lat, row.lng
    point_wkt = f"POINT({lon} {lat})"
    
    for year in np.arange(2000, 2023):
        print(f"Year: {year}")
        payload = {
            "wkt": point_wkt,
            "names": str(year),
            "interval": 5,
            "utc": "true",
            "leap_day": "true",
            "attributes": "windspeed_100m,winddirection_100m",
            "email": "charlesgulian@berkeley.edu",
            "full_name": "Charles Gulian",
            "affiliation": "UC Berkeley IEOR",
            "reason": "Research",
            "mailing_list": "false"
        }
        
        # Send request
        print("Submitting WTK data request to NREL...")
        response = requests.post(url, data=payload, timeout=60)
        
        if response.status_code == 200:
            print("Request submitted successfully.")

            # Save CSV file
            fname = bus_wind_data_dir / f"bus{bus_id}-{year}.csv"
            with open(fname, "wb") as f:
                f.write(response.content)

            # Re-open CSV with pandas and clean
            df_wind = pd.read_csv(fname, skiprows=1) # Remove header
            df_wind.columns = [c.strip() for c in df_wind.columns] # Clean columns names
            df_wind["datetime"] = pd.to_datetime(df_wind[["Year", "Month", "Day", "Hour", "Minute"]]) # Create datetime column
            df_wind = df_wind[["datetime", "wind speed at 100m (m/s)", "wind direction at 100m (deg)"]].set_index("datetime") # Restrict to columns of interest + change index
            df_wind.index = pd.to_datetime(df_wind.index) # Convert to datetime index

            # Save file again
            df_wind.to_csv(fname)

            del response
            del df_wind
            
        else:
            print(f"Request failed with status {response.status_code}")
            print(response.text)
            break
    print("Done.")

Retrieving wind data for bus 309...
Year: 2000
Submitting WTK data request to NREL...
Request submitted successfully.
Year: 2001
Submitting WTK data request to NREL...
Request submitted successfully.
Year: 2002
Submitting WTK data request to NREL...
Request submitted successfully.
Year: 2003
Submitting WTK data request to NREL...
Request submitted successfully.
Year: 2004
Submitting WTK data request to NREL...
Request submitted successfully.
Year: 2005
Submitting WTK data request to NREL...
Request submitted successfully.
Year: 2006
Submitting WTK data request to NREL...
Request submitted successfully.
Year: 2007
Submitting WTK data request to NREL...
Request submitted successfully.
Year: 2008
Submitting WTK data request to NREL...
Request submitted successfully.
Year: 2009
Submitting WTK data request to NREL...
Request submitted successfully.
Year: 2010
Submitting WTK data request to NREL...
Request submitted successfully.
Year: 2011
Submitting WTK data request to NREL...
Request subm

In [1]:
fname

NameError: name 'fname' is not defined

In [ ]:
# Save CSV file
fname = bus_wind_data_dir / f"bus{bus_id}-{year}.csv"

# Re-open CSV with pandas and clean
df_wind = pd.read_csv(fname)